In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Load original clean quote data
df_quotes = pd.read_csv('cleaned_quote_data.csv')
df_quotes['dt_creation_devis'] = pd.to_datetime(df_quotes['dt_creation_devis'])


def enhanced_data_discovery_analysis(df_quotes, X_new_clean=None):
    """
    Comprehensive data discovery focusing on new segments and predictive signals
    Uses df_quotes for raw analysis, X_new_clean for feature analysis
    """
    print("=" * 80)
    print("ENHANCED DATA DISCOVERY ANALYSIS - UPGRADED DATASET")
    print("=" * 80)
    
    # Initialize results dictionary
    analysis_results = {}
    
    # ========== DATA STRUCTURE CHECK ==========
    print("\n📁 DATASET STRUCTURE ANALYSIS")
    print("-" * 60)
    print(f"• df_quotes shape: {df_quotes.shape}")
    print(f"• df_quotes columns: {len(df_quotes.columns)}")
    
    if X_new_clean is not None:
        print(f"• X_new_clean shape: {X_new_clean.shape}")
        print(f"• X_new_clean columns: {len(X_new_clean.columns)}")
    
    # Check for customer ID column
    customer_id_col = 'numero_compte'
    if customer_id_col not in df_quotes.columns:
        # Try to find alternative customer ID column
        possible_ids = ['customer_id', 'client_id', 'id_client', 'customer_number']
        for col in possible_ids:
            if col in df_quotes.columns:
                customer_id_col = col
                break
        
        if customer_id_col == 'numero_compte':
            print("❌ ERROR: No customer ID column found in df_quotes!")
            print("Available columns:", list(df_quotes.columns))
            return analysis_results
    
    print(f"✅ Using customer ID column: '{customer_id_col}'")
    print(f"✅ Unique customers in df_quotes: {df_quotes[customer_id_col].nunique():,}")
    
    # ========== 1. CUSTOMER SEGMENT ANALYSIS ==========
    print("\n\n📊 1. CUSTOMER SEGMENT ANALYSIS")
    print("-" * 60)
    
    # Check for client status column
    client_status_col = None
    possible_cols = ['client_status', 'statut_client', 'fg_statut_client', 'status_client']
    
    for col in possible_cols:
        if col in df_quotes.columns:
            client_status_col = col
            break
    
    if client_status_col:
        print(f"✅ Found client status column: '{client_status_col}'")
        
        # Segment distribution
        segment_counts = df_quotes[client_status_col].value_counts()
        segment_pct = df_quotes[client_status_col].value_counts(normalize=True) * 100
        
        print(f"\n📈 Client Segment Distribution:")
        for segment, count in segment_counts.items():
            pct = segment_pct[segment]
            print(f"   • {segment}: {count:,} quotes ({pct:.1f}%)")
        
        # Analyze conversion rates by segment
        if 'fg_devis_accepte' in df_quotes.columns:
            print(f"\n🎯 Conversion Rates by Segment:")
            conversion_by_segment = df_quotes.groupby(client_status_col)['fg_devis_accepte'].mean() * 100
            
            for segment, conv_rate in conversion_by_segment.items():
                print(f"   • {segment}: {conv_rate:.1f}% conversion rate")
            
            analysis_results['segment_conversion_rates'] = conversion_by_segment
        
        # Analyze quote patterns by segment
        print(f"\n📝 Quote Patterns by Segment:")
        quotes_by_segment = df_quotes.groupby(client_status_col)[customer_id_col].agg(['count', 'nunique'])
        quotes_by_segment['avg_quotes_per_customer'] = quotes_by_segment['count'] / quotes_by_segment['nunique']
        
        print(f"   Quote statistics by segment:")
        for segment in quotes_by_segment.index:
            total_quotes = quotes_by_segment.loc[segment, 'count']
            unique_customers = quotes_by_segment.loc[segment, 'nunique']
            avg_quotes = quotes_by_segment.loc[segment, 'avg_quotes_per_customer']
            print(f"   • {segment}: {total_quotes:,} quotes, {unique_customers:,} customers ({avg_quotes:.2f} quotes/customer)")
    else:
        print("❌ No client status column found in df_quotes")
    
    # ========== 2. ENGAGEMENT SIGNAL ANALYSIS ==========
    print("\n\n📈 2. ENGAGEMENT SIGNAL ANALYSIS")
    print("-" * 60)
    
    # Check date column for temporal analysis
    date_col = 'dt_creation_devis'
    if date_col in df_quotes.columns:
        df_quotes[date_col] = pd.to_datetime(df_quotes[date_col], errors='coerce')
        
        # Engagement frequency analysis
        engagement_stats = df_quotes.groupby(customer_id_col).agg(
            total_quotes=(customer_id_col, 'size'),
            engagement_days=(date_col, lambda x: (x.max() - x.min()).days if len(x) > 1 else 0)
        ).reset_index()
        
        # Calculate quotes per day for customers with multiple quotes
        engagement_stats['quotes_per_day'] = engagement_stats.apply(
            lambda x: x['total_quotes'] / max(x['engagement_days'], 1), axis=1
        )
        
        print("📊 Customer Engagement Patterns:")
        print(f"   • Average quotes per customer: {engagement_stats['total_quotes'].mean():.2f}")
        print(f"   • Median quotes per customer: {engagement_stats['total_quotes'].median():.1f}")
        print(f"   • Max quotes for a single customer: {engagement_stats['total_quotes'].max()}")
        print(f"   • Average engagement days: {engagement_stats[engagement_stats['engagement_days'] > 0]['engagement_days'].mean():.1f}")
        
        # Single vs multi-quote analysis
        single_quote_customers = (engagement_stats['total_quotes'] == 1).sum()
        multi_quote_customers = (engagement_stats['total_quotes'] > 1).sum()
        total_customers = len(engagement_stats)
        
        print(f"\n🔢 Single vs Multi-Quote Customers:")
        print(f"   • Single-quote customers: {single_quote_customers:,} ({single_quote_customers/total_customers*100:.1f}%)")
        print(f"   • Multi-quote customers: {multi_quote_customers:,} ({multi_quote_customers/total_customers*100:.1f}%)")
        
        if 'fg_devis_accepte' in df_quotes.columns:
            # Get customer-level conversion
            customer_conversion = df_quotes.groupby(customer_id_col)['fg_devis_accepte'].max().reset_index()
            engagement_stats = engagement_stats.merge(customer_conversion, on=customer_id_col)
            
            # Conversion rates by engagement pattern
            conversion_single = engagement_stats[engagement_stats['total_quotes'] == 1]['fg_devis_accepte'].mean() * 100
            conversion_multi = engagement_stats[engagement_stats['total_quotes'] > 1]['fg_devis_accepte'].mean() * 100
            
            print(f"\n🎯 Conversion by Engagement Pattern:")
            print(f"   • Single-quote customers: {conversion_single:.1f}% conversion")
            print(f"   • Multi-quote customers: {conversion_multi:.1f}% conversion")
            
            if conversion_single > conversion_multi:
                print(f"   ⚡ INSIGHT: Single-quote customers convert {conversion_single - conversion_multi:.1f}% HIGHER!")
            else:
                print(f"   ⚡ INSIGHT: Multi-quote customers convert {conversion_multi - conversion_single:.1f}% HIGHER!")
            
            analysis_results['single_vs_multi_conversion'] = {
                'single': conversion_single,
                'multi': conversion_multi,
                'difference': abs(conversion_single - conversion_multi)
            }
    else:
        print("❌ No date column found for engagement analysis")
    
    # ========== 3. PRICE SIGNAL ANALYSIS ==========
    print("\n\n💰 3. PRICE SIGNAL ANALYSIS")
    print("-" * 60)
    
    price_cols = ['total_price', 'montant_total', 'prix_total', 'montant_devis', 'montant_ttc']
    price_col = None
    
    for col in price_cols:
        if col in df_quotes.columns and df_quotes[col].notna().sum() > 0:
            price_col = col
            break
    
    if price_col:
        # Clean price data
        df_quotes[price_col] = pd.to_numeric(df_quotes[price_col], errors='coerce')
        
        print(f"✅ Found price column: '{price_col}'")
        print(f"\n📊 Price Statistics (all quotes):")
        print(f"   • Mean price: €{df_quotes[price_col].mean():,.0f}")
        print(f"   • Median price: €{df_quotes[price_col].median():,.0f}")
        print(f"   • Min price: €{df_quotes[price_col].min():,.0f}")
        print(f"   • Max price: €{df_quotes[price_col].max():,.0f}")
        print(f"   • Price std: €{df_quotes[price_col].std():,.0f}")
        print(f"   • Missing values: {df_quotes[price_col].isna().sum():,} ({df_quotes[price_col].isna().mean()*100:.1f}%)")
        
        if 'fg_devis_accepte' in df_quotes.columns:
            # Price analysis by conversion
            print(f"\n🎯 Price Analysis by Conversion Status:")
            
            converted_prices = df_quotes[df_quotes['fg_devis_accepte'] == 1][price_col]
            non_converted_prices = df_quotes[df_quotes['fg_devis_accepte'] == 0][price_col]
            
            print(f"   • Converted quotes: €{converted_prices.mean():,.0f} (median: €{converted_prices.median():,.0f})")
            print(f"   • Non-converted quotes: €{non_converted_prices.mean():,.0f} (median: €{non_converted_prices.median():,.0f})")
            
            # Price quartile analysis
            valid_prices = df_quotes[price_col].dropna()
            if len(valid_prices) > 0:
                df_quotes['price_quartile'] = pd.qcut(df_quotes[price_col], q=4, 
                                                     labels=['Q1 (lowest)', 'Q2', 'Q3', 'Q4 (highest)'], 
                                                     duplicates='drop')
                quartile_conversion = df_quotes.groupby('price_quartile')['fg_devis_accepte'].mean() * 100
                
                print(f"\n📈 Conversion by Price Quartile:")
                for quartile, rate in quartile_conversion.items():
                    print(f"   • {quartile}: {rate:.1f}% conversion")
                
                # Find optimal price range
                best_quartile = quartile_conversion.idxmax()
                best_rate = quartile_conversion.max()
                print(f"   ⚡ OPTIMAL: {best_quartile} has highest conversion at {best_rate:.1f}%")
                
                analysis_results['price_quartile_conversion'] = quartile_conversion
    else:
        print("❌ No price column found for price analysis")
    
    # ========== 4. BRAND SIGNAL ANALYSIS ==========
    print("\n\n🏷️ 4. BRAND SIGNAL ANALYSIS")
    print("-" * 60)
    
    brand_cols = ['brand', 'marque', 'nom_marque', 'marque_produit', 'libelle_marque']
    brand_col = None
    
    for col in brand_cols:
        if col in df_quotes.columns and df_quotes[col].notna().sum() > 0:
            brand_col = col
            break
    
    if brand_col:
        print(f"✅ Found brand column: '{brand_col}'")
        
        # Clean brand names
        df_quotes[brand_col] = df_quotes[brand_col].astype(str).str.strip().str.upper()
        
        # Brand distribution
        brand_counts = df_quotes[brand_col].value_counts()
        brand_pct = df_quotes[brand_col].value_counts(normalize=True) * 100
        
        print(f"\n🏆 Top 10 Brands by Quote Volume:")
        top_brands = brand_counts.head(10)
        
        for i, (brand, count) in enumerate(top_brands.items(), 1):
            pct = brand_pct[brand]
            print(f"   {i:2d}. {brand:25} {count:6,} quotes ({pct:.1f}% share)")
        
        if 'fg_devis_accepte' in df_quotes.columns:
            # Brand conversion analysis
            print(f"\n🎯 Top 10 Brands by Conversion Rate (min 50 quotes):")
            
            # Filter brands with sufficient data
            brand_stats = df_quotes.groupby(brand_col).agg(
                quote_count=('fg_devis_accepte', 'count'),
                conversion_rate=('fg_devis_accepte', 'mean')
            )
            
            # Get top converting brands with enough quotes
            qualifying_brands = brand_stats[brand_stats['quote_count'] >= 50]
            
            if len(qualifying_brands) > 0:
                top_converting = qualifying_brands.nlargest(10, 'conversion_rate')
                
                for i, (brand, stats) in enumerate(top_converting.iterrows(), 1):
                    conv_rate = stats['conversion_rate'] * 100
                    count = int(stats['quote_count'])
                    print(f"   {i:2d}. {brand:25} {conv_rate:5.1f}% conversion ({count:,} quotes)")
                
                # Low converting brands
                print(f"\n⚠️  Bottom 5 Converting Brands (min 50 quotes):")
                low_converting = qualifying_brands.nsmallest(5, 'conversion_rate')
                
                for i, (brand, stats) in enumerate(low_converting.iterrows(), 1):
                    conv_rate = stats['conversion_rate'] * 100
                    count = int(stats['quote_count'])
                    print(f"   {i:2d}. {brand:25} {conv_rate:5.1f}% conversion ({count:,} quotes)")
                
                analysis_results['brand_conversion_stats'] = qualifying_brands
            else:
                print("   No brands have sufficient data for conversion analysis")
    else:
        print("❌ No brand column found for brand analysis")
    
    # ========== 5. EQUIPMENT SIGNAL ANALYSIS ==========
    print("\n\n⚙️ 5. EQUIPMENT SIGNAL ANALYSIS")
    print("-" * 60)
    
    equipment_cols = ['regroup_famille_equipement_produit', 'famille_equipement_produit', 
                      'equipment_type', 'type_equipement', 'libelle_equipement']
    equipment_col = None
    
    for col in equipment_cols:
        if col in df_quotes.columns and df_quotes[col].notna().sum() > 0:
            equipment_col = col
            break
    
    if equipment_col:
        print(f"✅ Found equipment column: '{equipment_col}'")
        
        # Clean equipment names
        df_quotes[equipment_col] = df_quotes[equipment_col].astype(str).str.strip()
        
        # Equipment distribution
        equipment_counts = df_quotes[equipment_col].value_counts()
        equipment_pct = df_quotes[equipment_col].value_counts(normalize=True) * 100
        
        print(f"\n🔧 Top 10 Equipment Categories:")
        for i, (category, count) in enumerate(equipment_counts.head(10).items(), 1):
            pct = equipment_pct[category]
            print(f"   {i:2d}. {category:35} {count:6,} quotes ({pct:.1f}%)")
        
        if 'fg_devis_accepte' in df_quotes.columns:
            # Equipment conversion analysis
            print(f"\n🎯 Equipment Conversion Analysis (min 50 quotes):")
            
            equipment_stats = df_quotes.groupby(equipment_col).agg(
                quote_count=('fg_devis_accepte', 'count'),
                conversion_rate=('fg_devis_accepte', 'mean')
            )
            
            # Get top converting equipment categories
            qualifying_equipment = equipment_stats[equipment_stats['quote_count'] >= 50]
            
            if len(qualifying_equipment) > 0:
                top_equipment = qualifying_equipment.nlargest(8, 'conversion_rate')
                
                for i, (category, stats) in enumerate(top_equipment.iterrows(), 1):
                    conv_rate = stats['conversion_rate'] * 100
                    count = int(stats['quote_count'])
                    print(f"   {i:2d}. {category:35} {conv_rate:5.1f}% conversion ({count:,} quotes)")
                
                analysis_results['equipment_conversion_stats'] = equipment_stats
            else:
                print("   No equipment categories have sufficient data for conversion analysis")
    else:
        print("❌ No equipment column found for equipment analysis")
    
    # ========== 6. PROCESS ADOPTION ANALYSIS ==========
    print("\n\n🔄 6. PROCESS ADOPTION ANALYSIS")
    print("-" * 60)
    
    process_cols = ['fg_nouveau_process_relance_devis', 'nouveau_process', 'process_type']
    process_col = None
    
    for col in process_cols:
        if col in df_quotes.columns:
            process_col = col
            break
    
    if process_col:
        print(f"✅ Found process column: '{process_col}'")
        
        process_dist = df_quotes[process_col].value_counts(normalize=True) * 100
        
        print(f"\n📊 Process Distribution:")
        total_quotes = len(df_quotes)
        for process, count in df_quotes[process_col].value_counts().items():
            pct = count / total_quotes * 100
            if pd.isna(process):
                print(f"   • Missing/Unknown: {count:,} quotes ({pct:.1f}%)")
            else:
                process_name = "New Process" if process == 1 else "Old Process"
                print(f"   • {process_name}: {count:,} quotes ({pct:.1f}%)")
        
        if 'fg_devis_accepte' in df_quotes.columns:
            print(f"\n🎯 Process Conversion Analysis:")
            process_conversion = df_quotes.groupby(process_col)['fg_devis_accepte'].mean() * 100
            
            for process, conv_rate in process_conversion.items():
                if pd.isna(process):
                    process_name = "Missing/Unknown"
                else:
                    process_name = "New Process" if process == 1 else "Old Process"
                print(f"   • {process_name}: {conv_rate:.1f}% conversion")
            
            # Calculate improvement
            if 1 in process_conversion.index and 0 in process_conversion.index:
                new_rate = process_conversion[1]
                old_rate = process_conversion[0]
                improvement = new_rate - old_rate
                print(f"   ⚡ IMPACT: New process shows {improvement:+.1f}% conversion change")
            
            analysis_results['process_conversion'] = process_conversion
    else:
        print("❌ No process column found for process analysis")
    
    # ========== 7. COMMERCIAL ROLE ANALYSIS ==========
    print("\n\n👥 7. COMMERCIAL ROLE ANALYSIS")
    print("-" * 60)
    
    role_cols = ['fonction_commercial', 'commercial_role', 'role_commercial']
    role_col = None
    
    for col in role_cols:
        if col in df_quotes.columns:
            role_col = col
            break
    
    if role_col:
        print(f"✅ Found commercial role column: '{role_col}'")
        
        role_counts = df_quotes[role_col].value_counts()
        
        print(f"\n📊 Top 10 Commercial Roles:")
        for i, (role, count) in enumerate(role_counts.head(10).items(), 1):
            pct = count / len(df_quotes) * 100
            print(f"   {i:2d}. {role:30} {count:6,} quotes ({pct:.1f}%)")
        
        if 'fg_devis_accepte' in df_quotes.columns:
            print(f"\n🎯 Top Converting Roles (min 30 quotes):")
            
            role_stats = df_quotes.groupby(role_col).agg(
                quote_count=('fg_devis_accepte', 'count'),
                conversion_rate=('fg_devis_accepte', 'mean')
            )
            
            # Filter for significant roles
            significant_roles = role_stats[role_stats['quote_count'] >= 30]
            
            if len(significant_roles) > 0:
                top_roles = significant_roles.nlargest(10, 'conversion_rate')
                
                for i, (role, stats) in enumerate(top_roles.iterrows(), 1):
                    conv_rate = stats['conversion_rate'] * 100
                    count = int(stats['quote_count'])
                    print(f"   {i:2d}. {role:30} {conv_rate:5.1f}% conversion ({count:,} quotes)")
                
                analysis_results['role_conversion_stats'] = significant_roles
            else:
                print("   No roles have sufficient data for conversion analysis")
    else:
        print("❌ No commercial role column found")
    
    # ========== 8. AGENCY PERFORMANCE ANALYSIS ==========
    print("\n\n🏢 8. AGENCY PERFORMANCE ANALYSIS")
    print("-" * 60)
    
    agency_cols = ['code_agence', 'agence', 'nom_agence', 'libelle_agence']
    agency_col = None
    
    for col in agency_cols:
        if col in df_quotes.columns:
            agency_col = col
            break
    
    if agency_col:
        print(f"✅ Found agency column: '{agency_col}'")
        
        agency_counts = df_quotes[agency_col].value_counts()
        
        print(f"\n📊 Top 10 Agencies by Quote Volume:")
        for i, (agency, count) in enumerate(agency_counts.head(10).items(), 1):
            pct = count / len(df_quotes) * 100
            print(f"   {i:2d}. {agency:25} {count:6,} quotes ({pct:.1f}%)")
        
        if 'fg_devis_accepte' in df_quotes.columns:
            print(f"\n🎯 Agency Conversion Performance (min 100 quotes):")
            
            agency_stats = df_quotes.groupby(agency_col).agg(
                quote_count=('fg_devis_accepte', 'count'),
                conversion_rate=('fg_devis_accepte', 'mean')
            )
            
            if price_col:
                agency_stats['avg_price'] = df_quotes.groupby(agency_col)[price_col].mean()
            
            # Filter for significant agencies
            significant_agencies = agency_stats[agency_stats['quote_count'] >= 100]
            
            if len(significant_agencies) > 0:
                print(f"\n🏆 Top 5 Converting Agencies:")
                top_agencies = significant_agencies.nlargest(5, 'conversion_rate')
                for i, (agency, stats) in enumerate(top_agencies.iterrows(), 1):
                    conv_rate = stats['conversion_rate'] * 100
                    count = int(stats['quote_count'])
                    avg_price = f"€{stats['avg_price']:,.0f}" if price_col and 'avg_price' in stats else "N/A"
                    print(f"   {i:2d}. {agency:20} {conv_rate:5.1f}% conversion, {count:5,} quotes, avg: {avg_price}")
                
                print(f"\n⚠️  Bottom 5 Converting Agencies:")
                bottom_agencies = significant_agencies.nsmallest(5, 'conversion_rate')
                for i, (agency, stats) in enumerate(bottom_agencies.iterrows(), 1):
                    conv_rate = stats['conversion_rate'] * 100
                    count = int(stats['quote_count'])
                    avg_price = f"€{stats['avg_price']:,.0f}" if price_col and 'avg_price' in stats else "N/A"
                    print(f"   {i:2d}. {agency:20} {conv_rate:5.1f}% conversion, {count:5,} quotes, avg: {avg_price}")
                
                analysis_results['agency_stats'] = significant_agencies
            else:
                print("   No agencies have sufficient data for conversion analysis")
    else:
        print("❌ No agency column found for agency analysis")
    
    # ========== 9. FEATURE DATASET ANALYSIS (X_new_clean) ==========
    if X_new_clean is not None:
        print("\n\n📊 9. FEATURE DATASET ANALYSIS (X_new_clean)")
        print("-" * 60)
        
        print(f"Feature dataset shape: {X_new_clean.shape}")
        print(f"Feature types distribution:")
        
        # Count feature types
        numeric_features = X_new_clean.select_dtypes(include=[np.number]).columns.tolist()
        categorical_features = X_new_clean.select_dtypes(include=['object', 'category']).columns.tolist()
        bool_features = X_new_clean.select_dtypes(include=['bool']).columns.tolist()
        
        print(f"   • Numeric features: {len(numeric_features)}")
        print(f"   • Categorical features: {len(categorical_features)}")
        print(f"   • Boolean features: {len(bool_features)}")
        
        # Show top features by variance (for numeric)
        if len(numeric_features) > 0:
            print(f"\n📈 Top 10 Numeric Features by Variance:")
            variances = X_new_clean[numeric_features].var().sort_values(ascending=False)
            for i, (feature, variance) in enumerate(variances.head(10).items(), 1):
                print(f"   {i:2d}. {feature:35} variance: {variance:.4f}")
        
        # Show top categorical features by unique values
        if len(categorical_features) > 0:
            print(f"\n🏷️  Categorical Features Cardinality:")
            unique_counts = X_new_clean[categorical_features].nunique().sort_values(ascending=False)
            for i, (feature, count) in enumerate(unique_counts.head(10).items(), 1):
                print(f"   {i:2d}. {feature:35} unique values: {count}")
    
    # ========== 10. SUMMARY & INSIGHTS ==========
    print("\n\n📋 10. DATA QUALITY & INSIGHTS SUMMARY")
    print("=" * 60)
    
    # Data quality metrics
    print(f"\n📊 Dataset Overview:")
    print(f"   • Total quotes in df_quotes: {len(df_quotes):,}")
    print(f"   • Unique customers: {df_quotes[customer_id_col].nunique():,}")
    
    if 'fg_devis_accepte' in df_quotes.columns:
        conversion_rate = df_quotes['fg_devis_accepte'].mean() * 100
        print(f"   • Overall conversion rate: {conversion_rate:.1f}%")
    
    # Missing data analysis
    print(f"\n🔍 Missing Data Analysis (df_quotes):")
    missing_data = df_quotes.isnull().sum()
    missing_pct = (missing_data / len(df_quotes)) * 100
    
    # Show top columns with missing data
    missing_summary = pd.DataFrame({
        'missing_count': missing_data,
        'missing_percent': missing_pct
    }).sort_values('missing_percent', ascending=False)
    
    print("   Top columns with missing values:")
    for i, (col, row) in enumerate(missing_summary.head(10).iterrows(), 1):
        if row['missing_percent'] > 0:
            print(f"   {i:2d}. {col:35} {row['missing_count']:,} missing ({row['missing_percent']:.1f}%)")
    
    # Generate insights
    print(f"\n💡 KEY INSIGHTS:")
    
    insights = []
    
    # Insight 1: Single-quote conversion advantage
    if 'single_vs_multi_conversion' in analysis_results:
        single_conv = analysis_results['single_vs_multi_conversion']['single']
        multi_conv = analysis_results['single_vs_multi_conversion']['multi']
        diff = analysis_results['single_vs_multi_conversion']['difference']
        
        if single_conv > multi_conv:
            insights.append(f"• Single-quote customers convert {diff:.1f}% HIGHER than multi-quote customers")
        else:
            insights.append(f"• Multi-quote customers convert {diff:.1f}% HIGHER than single-quote customers")
    
    # Insight 2: Price-conversion relationship
    if 'price_quartile_conversion' in analysis_results:
        quartiles = analysis_results['price_quartile_conversion']
        if len(quartiles) >= 2:
            best_q = quartiles.idxmax()
            worst_q = quartiles.idxmin()
            best_rate = quartiles.max()
            worst_rate = quartiles.min()
            
            insights.append(f"• {best_q} quotes convert {best_rate-worst_rate:.1f}% better than {worst_q} quotes")
    
    # Insight 3: Process effectiveness
    if 'process_conversion' in analysis_results:
        process_rates = analysis_results['process_conversion']
        if 1 in process_rates.index and 0 in process_rates.index:
            new_rate = process_rates[1]
            old_rate = process_rates[0]
            if not pd.isna(new_rate) and not pd.isna(old_rate):
                improvement = new_rate - old_rate
                if improvement > 0:
                    insights.append(f"• New process converts {improvement:.1f}% BETTER than old process")
                elif improvement < 0:
                    insights.append(f"• New process converts {abs(improvement):.1f}% WORSE than old process")
    
    # Print insights
    if insights:
        for i, insight in enumerate(insights, 1):
            print(f"   {i}. {insight}")
    else:
        print("   No clear predictive patterns identified from basic analysis")
    
    return analysis_results

# ========== MAIN EXECUTION ==========
if __name__ == "__main__":
    # Run enhanced analysis using your datasets
    print("🚀 STARTING ENHANCED DATA DISCOVERY")
    print("=" * 80)
    
    # Run the main analysis
    analysis_results = enhanced_data_discovery_analysis(
        df_quotes=df_quotes,  # Your raw dataset with customer IDs
    )
    
    print("\n" + "=" * 80)
    print("ANALYSIS COMPLETE - READY FOR FEATURE ENGINEERING OPTIMIZATION")
    print("=" * 80)

🚀 STARTING ENHANCED DATA DISCOVERY
ENHANCED DATA DISCOVERY ANALYSIS - UPGRADED DATASET

📁 DATASET STRUCTURE ANALYSIS
------------------------------------------------------------
• df_quotes shape: (34014, 47)
• df_quotes columns: 47
✅ Using customer ID column: 'numero_compte'
✅ Unique customers in df_quotes: 23,888


📊 1. CUSTOMER SEGMENT ANALYSIS
------------------------------------------------------------
✅ Found client status column: 'statut_client'

📈 Client Segment Distribution:
   • NEW_PROSPECT: 25,209 quotes (74.1%)
   • OLD_PROSPECT: 4,866 quotes (14.3%)
   • ACTIVE_CUSTOMER: 3,939 quotes (11.6%)

🎯 Conversion Rates by Segment:
   • ACTIVE_CUSTOMER: 35.5% conversion rate
   • NEW_PROSPECT: 28.6% conversion rate
   • OLD_PROSPECT: 32.6% conversion rate

📝 Quote Patterns by Segment:
   Quote statistics by segment:
   • ACTIVE_CUSTOMER: 3,939 quotes, 2,703 customers (1.46 quotes/customer)
   • NEW_PROSPECT: 25,209 quotes, 18,286 customers (1.38 quotes/customer)
   • OLD_PROSPECT: